In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [27]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
import matplotlib.pyplot as plt
import os
import requests
from bs4 import BeautifulSoup
import urllib
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
import psycopg2
workers = ipp.Client()[:]
plt.rcParams['figure.figsize'] = [30/2.54, 20/2.54]
PART_SIZE = int(10**6 / 2)
ENV_URL = "http://192.168.0.6/env.json" 
HOME = ""
workers

In [28]:
def make_cur():
    resp = requests.get(url=ENV_URL)
    envdata = resp.json()["pressure"]

    print(envdata)
    con = psycopg2.connect(
        database=envdata["database"],
        user=envdata["user"],
        password=envdata["password"],
        host=envdata["host"],
        port=envdata["port"]

    )
    cur = con.cursor()
    if not con or not cur:
        print("no con")
        exit(1)
    return con, cur

def get_count() -> int:
    con, cur = make_cur()
    cur.execute("SELECT count(id) FROM public.pressure;")
    count = cur.fetchone()[0]
    mprint(count)
    con.close()
    return count

In [49]:
def extractPart(job):
    import sys
    import pandas as pd
    import numpy as np
    import psycopg2
    import matplotlib
    import matplotlib.pyplot as plt
    import os
    import requests
    
    log = []
    file = open("job%d.log" % os.getpid(), "w")
    file.write(str(job) + "\n")
    df = "df"
    def mprint(msg):
        print(msg)
        log.append(str(msg))
        file.write(str(msg) + "\n")
        file.flush()
        sys.stdout.flush()
        
    start, count, path = job
    mprint(job)
    try:
        def make_cur():

            return con, cur


        con, cur = make_cur()
        if not con or not cur:
            return None, log

        mprint("write %d-%d to %s" % (start, start + PART_SIZE, path))
        q = "SELECT * FROM public.pressure ORDER BY dt ASC LIMIT %d OFFSET %d;" % (count, start)
        cur.execute(q)
        columns = ['i','Date', 'Pressure']

        idList = []
        pressureList = []
        dtList = []
        columns = ["id", "temperature", "pressure", "altitude", "realaltitude", "dt"]
        df = pd.DataFrame.from_records(cur.fetchall(), columns = columns)
        mprint(df.head())
        df.set_index(columns[0])
        con.close()
        df.to_csv(path_or_buf = (path + ".csv"), columns=columns, header=True, mode="w", index=False)
        df.plot(x="dt", y=["pressure"], figsize=(150, 90))
        plt.savefig(fname = (path + ".png"), format = "png")
    except exception as e:
        mprint(e)
    
    if file:
        file.close()
    return (df, log)

In [125]:
def test(job):
    import sys
    import pandas as pd
    import numpy as np
    import psycopg2
    import matplotlib
    import matplotlib.pyplot as plt
    import os
    import requests
    log = []
    ENV_URL = "http://192.168.0.6/env.json" 
    df = "df"
    def mprint(msg):
        m = "[%d] - %s" % (os.getpid(), str(msg))
        print(m)
        log.append(m)
        sys.stdout.flush()

    start, count, path = job
    resp = requests.get(url=ENV_URL)
    envdata = resp.json()["pressure"]
    con = psycopg2.connect(
        database=envdata["database"],
        user=envdata["user"],
        password=envdata["password"],
        host=envdata["host"],
        port=envdata["port"]

    )
    if not con:
        mprint("no con")
        return log, None
    cur = con.cursor()
    if not cur:
        mprint("no cur")
        return log, None
    mprint("connected")
    mprint("write %d-%d to %s" % (start, start + count, path))
    q = "SELECT * FROM public.pressure ORDER BY dt ASC LIMIT %d OFFSET %d" % (count, start)
    mprint(q)
    #mprint(сur.statusmessage)
    #try:
    #    cur.execute(q)
    #except exception as e:
    #    mprint(e)
    columns = ['i','Date', 'Pressure']
    con.close()
    return (log, df)

In [126]:
jobs = []
count = 5000
for start in range(0, count, 1000):
    jobs.append((start, count, ("pressure_%d-%d" % (start, start + PART_SIZE))))
    jobs.append((start, count, ("pressure_%d-%d" % (start, start + PART_SIZE))))
    ##extractPart((start, PART_SIZE, ("pressure_%d-%d" % (start, start + PART_SIZE))))
    
##parts = workers.map_async(extractPart, jobs).get()
parts = workers.map(test, jobs)
parts.wait()
d = parts.get()
for joblist in list(zip(*d))[0]:
    print("----------------------------------------")
    for msg in joblist:
        print(msg)

----------------------------------------
[485] - connected
[485] - write 0-5000 to pressure_0-500000
[485] - SELECT * FROM public.pressure ORDER BY dt ASC LIMIT 5000 OFFSET 0
----------------------------------------
[485] - connected
[485] - write 0-5000 to pressure_0-500000
[485] - SELECT * FROM public.pressure ORDER BY dt ASC LIMIT 5000 OFFSET 0
----------------------------------------
[487] - connected
[487] - write 1000-6000 to pressure_1000-501000
[487] - SELECT * FROM public.pressure ORDER BY dt ASC LIMIT 5000 OFFSET 1000
----------------------------------------
[487] - connected
[487] - write 1000-6000 to pressure_1000-501000
[487] - SELECT * FROM public.pressure ORDER BY dt ASC LIMIT 5000 OFFSET 1000
----------------------------------------
[489] - connected
[489] - write 2000-7000 to pressure_2000-502000
[489] - SELECT * FROM public.pressure ORDER BY dt ASC LIMIT 5000 OFFSET 2000
----------------------------------------
[491] - connected
[491] - write 2000-7000 to pressure_200